In [5]:
##Python .py code from .jpynb:
from pyspark.sql.functions import rand, col
from pyspark.sql.functions import lower
from pyspark.sql.types import IntegerType, StringType, MapType
from pyspark.sql.functions import lit
from pyspark.sql.functions import row_number, lit, dense_rank, concat_ws
from pyspark.sql import functions as F, Window
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

#############################################
#            read data                      # 
#############################################
print("Read Anobii data")

# Read table which contains for each item_id (id of each book), the id of the author (author_id) who wrote that book
file = "/data/SMARTDATA/books/anobii_2021/sql/author_item.csv"
DFauthorbooks = spark.read.csv(file,header=True) #MOSTRA I LIBRI RELATIVI AGLI AUTORI

# Read table which contains for each author_id (id of each auhtor), the info of that author
file = "/data/SMARTDATA/books/anobii_2021/sql/author_display.csv"
DFdisplay = spark.read.csv(file,header=True) #MOSTRA GLI AUTORI DEL DATABASE

# Read table which contains the mapping of the language (language=11 means italian)
file = "/data/SMARTDATA/books/anobii_2021/sql/language_mapping.csv"
DFlanguages = spark.read.csv(file,header=True)

# Read table which contains the info about items, i.e. books
file = "/data/SMARTDATA/books/anobii_2021/sql/item.csv"
DFitems_anobii = spark.read.csv(file,header=True) 
# DFitems_anobii.show(1, False)

# Read table which contains the rate given by a person to a book
file = "/data/SMARTDATA/books/anobii_2021/sql/link_person_item.csv" #item_review ha il voto dato dall'utente a quel libro
DFinfo = spark.read.csv(file,header=True)
# DFinfo.show(1, False)

# Read table which contains the genre of each book
file = "anobii_genres/new_genres5.csv"
DFgenres = spark.read.csv(file, header=True)
#DFgenres.filter(DFgenres.itemid == "1981748").show(10, False)

file = "anobii_genres/content.csv"
DFdescriptions = spark.read.csv(file, header=True)
#DFdescriptions.show(1, False)

#DFdescriptions.filter(DFdescriptions.item_id=="2015441").show(2, False)
#FUNZIONI
file = "/data/SMARTDATA/books/anobii/link_person_item_comment.csv"
DFreviews = spark.read.csv(file, header=True)
#DFreviews.show(1, False)

def title_and_subtitle(line): #Usata per splittare titolo e sottotitolo nel database delle biblioteche e renderli più simili ad anobii
    if len(line.title.split(" : ")) > 1: #Il libro possiede un sottotitolo
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
        return Row(line.manifestation_id_new, line.patron_id_md5, title, sub_title, line.author, 4, "None", 4, "None", line.ISBNISSN_new, "bct") #NE APPROFITTO ANCHE PER INSERIRE VOTO E CAMPI PER IL MERGE (PER ORA NULL)
    else: #Il libro non possiede un sottotitolo
        return Row(line.manifestation_id_new, line.patron_id_md5, line.title, "None", line.author, 4, "None", 4, "None", line.ISBNISSN_new, "bct")

def create_user_dictionary(rdd): #Used to assign an integer id to each user of the rdd (to get the rows of the CSR)
    rdd = rdd.map(lambda x: (str(x.person_id), list(x))).sortByKey()
    user_dictionary = rdd.countByKey()
    i = 0
    for key in user_dictionary.keys():
        user_dictionary[key] = i
        i += 1
    return user_dictionary

def create_book_dictionary(rdd): #Used to assign an integer id to each book of the rdd (to get the column of the CSR)
    rdd = rdd.map(lambda x: (x.book_id, list(x))).sortByKey()
    book_dictionary = rdd.countByKey()
    i = 0
    for key in book_dictionary.keys():
        book_dictionary[key] = i
        i += 1
    return book_dictionary

def addDataType1(line):
    return Row(line.item_id, line.person_id, line.title, str(line.sub_title), line.item_review, line.total_review, line.average_rating, str(line.total_votes), line.isbn, "anobii")


def addRowColumnId(line): #Funzione usata per aggiungere gli indici per ogni libro e utente
    book_number = book_dictionary[line.book_id] 
    user_number = user_dictionary[line.person_id]
    return Row(line.book_id,
               line.title,
               line.sub_title,
               line.total_wishlist,
               line.no_of_page,
               line.publication_date,
               line.publisher,
               line.binding,
               line.edition,
               line.product_type,
               line.total_votes,
               line.data_type,
               line.person_id,
               line.item_review,
               line.author,
               line.genre,
               line.encrypt_item_id,
               line.isbn,
               line.total_count,
               line.average_rating,
               line.total_review,
               str(user_number), 
               str(book_number))
    return Row(line.person_id, line.title, str(line.sub_title), line.item_review, line.data_type, line.book_id, line.author, line.genre, line.encrypt_item_id, line.total_votes_or_loans, line.average_rating, line.total_review, line.isbn, str(user_number), str(book_number))

def title_and_subtitle_books(line): #FUNZIONE USATA PER DIVIDERE TITOLO E SOTTOTITOLO NEL DATASET DEI LIBRI DELLE BIBLIOTECHE E PER ELIMINARE EDITION_DATE ED EDITION_LANGUAGE
    if len(line.title.split(" : ")) > 1:
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
    else:
        title = line.title
        sub_title = "None"
    #"title", "sub_title", "author", "publisher", "book_id"
    return Row(title, sub_title, line.author, line.publisher, line.manifestation_id_new, line.ISBNISSN_new)

def concatenateContentsDescriptions(line1, line2):
    #print(line1)
    #Caso in cui vi è più di una descrizione presente per uno stesso libro
    return Row(book_id=line1.book_id, title=line1.title, sub_title=line1.sub_title, 
               total_wishlist=line1.total_wishlist, no_of_page=line1.no_of_page, publication_date=line1.publication_date, 
               publisher=line1.publisher, binding=line1.binding, edition=line1.edition, product_type=line1.product_type, 
               total_votes=line1.total_votes, data_type=line1.data_type, author=line1.author, genre=line1.genre, 
               encrypt_item_id=line1.encrypt_item_id, isbn=line1.isbn, total_count=line1.total_count, average_rating=line1.average_rating, 
               total_review=line1.total_review, book_index=line1.book_index, content=line1.content + " " + line2.content, content2=line1.content2)

def toSingleDescriptions(line1):
    #print(line1)
    return Row(book_id=line1[1].book_id, title=line1[1].title, sub_title=line1[1].sub_title, 
               total_wishlist=line1[1].total_wishlist, no_of_page=line1[1].no_of_page, publication_date=line1[1].publication_date, 
               publisher=line1[1].publisher, binding=line1[1].binding, edition=line1[1].edition, product_type=line1[1].product_type, 
               total_votes=line1[1].total_votes, data_type=line1[1].data_type, author=line1[1].author, genre=line1[1].genre, 
               encrypt_item_id=line1[1].encrypt_item_id, isbn=line1[1].isbn, total_count=line1[1].total_count, average_rating=line1[1].average_rating, 
               total_review=line1[1].total_review, book_index=line1[1].book_index, content=line1[1].content, content2=line1[1].content2)

def concatenateContentsComments(line1, line2):
    #print(line1)
    #Caso in cui dobbiamo concatenare i commenti di un libro alla sua descrizione
    return Row(book_id=line1.book_id, title=line1.title, sub_title=line1.sub_title, 
               total_wishlist=line1.total_wishlist, no_of_page=line1.no_of_page, publication_date=line1.publication_date, 
               publisher=line1.publisher, binding=line1.binding, edition=line1.edition, product_type=line1.product_type, 
               total_votes=line1.total_votes, data_type=line1.data_type, author=line1.author, genre=line1.genre, 
               encrypt_item_id=line1.encrypt_item_id, isbn=line1.isbn, total_count=line1.total_count, average_rating=line1.average_rating, 
               total_review=line1.total_review, book_index=line1.book_index, content=line1.content, content2=line1.content2, comment_content = str(line1.comment_content) + " " + str(line2.comment_content))

def mergeDescriptionComments(line1):
    return Row(book_id=line1.book_id, title=line1.title, sub_title=line1.sub_title, 
               total_wishlist=line1.total_wishlist, no_of_page=line1.no_of_page, publication_date=line1.publication_date, 
               publisher=line1.publisher, binding=line1.binding, edition=line1.edition, product_type=line1.product_type, 
               total_votes=line1.total_votes, data_type=line1.data_type, author=line1.author, genre=line1.genre, 
               encrypt_item_id=line1.encrypt_item_id, isbn=line1.isbn, total_count=line1.total_count, average_rating=line1.average_rating, 
               total_review=line1.total_review, book_index=line1.book_index, content2=line1.content2, content=str(line1.content) + " " + str(line1.comment_content))

def toSingleComments(line1):
    #print(line1)
    return Row(book_id=line1[1].book_id, title=line1[1].title, sub_title=line1[1].sub_title, 
               total_wishlist=line1[1].total_wishlist, no_of_page=line1[1].no_of_page, publication_date=line1[1].publication_date, 
               publisher=line1[1].publisher, binding=line1[1].binding, edition=line1[1].edition, product_type=line1[1].product_type, 
               total_votes=line1[1].total_votes, data_type=line1[1].data_type, author=line1[1].author, genre=line1[1].genre, 
               encrypt_item_id=line1[1].encrypt_item_id, isbn=line1[1].isbn, total_count=line1[1].total_count, average_rating=line1[1].average_rating, 
               total_review=line1[1].total_review, book_index=line1[1].book_index, content2=line1[1].content2, content=line1[1].content, comment_content = line1[1].comment_content)
print("Raws data have been read")
#these are the columns which are needed for our work, for the item table. The others can be dropped
DFitems_anobii_cols_to_keep = [
                        "item_id",\
                        "isbn",\
#                         "family_id",\
                        "title",\
                        "sub_title",\
                        # "barcode",\
                        # "image_source",\
                        # "image_width",\
                        # "image_height",\
                        "no_of_page",\
                        "publication_date",\
                        "publisher",\
                        "binding",\
                        "edition",\
                        # "reading_level",\
                        # "height",\
                        # "height_unit",\
                        # "length",\
                        # "length_unit",\
                        # "width",\
                        # "width_unit",\
                        # "weight",\
                        # "weight_unit",\
                        # "salesrank",\
                        # "item_popularity",\
                        # "check_same_family",\
                        # "check_internal_family",\
                        # "last_update",\
                        "average_rating",\
                        "total_review",\
                        "product_type",\
                        # "title_foreign",\
                        # "image_process",\
                        # "amazon",\
                        # "google",\
                        "encrypt_item_id",\
                        # "pid",\
                        # "binding_id",\
                        # "problem",\
                        "language",\
                        # "language_correct",\
                        # "ean",\
                        # "family_head",\
                        # "google_time",\
                        # "total_world",\
                        # "lock",\
                        # "volumes",\
                        # "publication_country_id",\
                        # "added_date",\
                        # "publishing_status",\
                        # "total_libraries",\
                        # "unavailable_date",\
                        # "table_of_contents_html",\
                        # "product_form_detail",\
                        # "total_edition_ratings",\
                        # "epub_url",\
                        # "imprint_name",\
                        # "is_sellable",\
                        # "total_topics",\
                        # "publisher_id",\
                        "total_wishlist",\
                        # "embargo_date",\
                        # "data_source",\
                        # "ebook_type",\
                        # "has_ebook",\
                        # "fulfillment_book_id",\
                        # "ebook_filesize",\
                        # "sample_status",\
                        # "sample_url",\
                        # "sample_filesize",\
                      ]
DFitems_anobii = DFitems_anobii.select(DFitems_anobii_cols_to_keep)
print("Filter Anobii data (select books in italian, drop comics and books with a low number of ratings)")
#FILTRAGGIO 1: linguaggio italiano
#Mi basterà filtrare da item.csv tutte le tuple con language diverso da 11.

DFfilteredlanguageitems = DFitems_anobii.filter(DFitems_anobii.language == "11")
DFitems_anobii = DFitems_anobii.select(DFitems_anobii_cols_to_keep)
#DFfilteredlanguageitems.show(20, False)

#FILTRAGGIO 2: solo "hardcover" e "paperback"

DFfilteredbindingitems = DFfilteredlanguageitems.filter((DFfilteredlanguageitems.binding == "Paperback") | (DFfilteredlanguageitems.binding == "Hardcover"))
#DFfilteredbindingitems.show(20, False)

#FILTRAGGIO 3: eliminare i fumetti (per quanto possibile)
#Userò come parole chiave personaggi famosi Disney quali Topolino e Paperino e i fumetti Bonelli (che fra gli italiani
#sono probabilmente i più gettonati)

DFfilteredmagazineitems = DFfilteredbindingitems.filter(~(DFfilteredbindingitems.title.contains("Topolino")) | (DFfilteredbindingitems.title.contains("Paperino"))|(DFfilteredbindingitems.title.contains("Tex"))|(DFfilteredbindingitems.title.contains("Dylan Dog"))|(DFfilteredbindingitems.title.contains("Nathan Never"))|(DFfilteredbindingitems.title.contains("Zagor")))

Read Anobii data
Raws data have been read
Filter Anobii data (select books in italian, drop comics and books with a low number of ratings)


In [2]:
#DFgenres.filter(DFgenres.votes.isNull()).show(1, False)

In [39]:
#PROCESSING DATAFRAME GENRES#

#Fase 1: eliminare le tuple con SelfHelp, Textbook e Reference come genere
#DFgenres.filter(DFgenres.name == "Textbook").show(1, False)
DFgenres = DFgenres.filter((DFgenres.name != "SelfHelp") & (DFgenres.name != "Reference") & (DFgenres.name != "Textbook"))

#Fase 1.2: eliminare le tuple con votes = null (esistono)

DFgenres = DFgenres.filter(~(DFgenres.votes.isNull()))
#DFgenres.filter(DFgenres.votes.isNull()).show(10, False)
#Fase 2: mappare i vari generi alla corrispondente aggregazione scelta da Greta e aggregare eventuali duplicati
#NOTA: usiamo un RDD

RDDgenres = DFgenres.rdd

def mapGenres(line):
    transformed_line = line.name
    if transformed_line == "Cooking-Food&Wine" or transformed_line == "Games" or transformed_line == "Crafts&Hobbies" or transformed_line == "Home&Gardening" or transformed_line == "Music" or transformed_line == "Art-Architecture&Photography" or transformed_line == "Sports-Outdoors&Adventure" or transformed_line == "Entertainment":
        transformed_line = "FreeTime"
    elif transformed_line == "Professional&Technical" or transformed_line == "Computer&Technology" or transformed_line == "Law" or transformed_line == "Medicine" or transformed_line == "Business&Economics" or transformed_line == "ForeignLanguageStudy" or transformed_line == "Education&Teaching":
        transformed_line = "Professional&Technical"
    elif transformed_line == "Health-Mind&Body" or transformed_line == "Religion&Spirituality":
        transformed_line = "Health-Mind&Body"
    elif transformed_line == "Family-Sex&Relationships" or transformed_line == "Gay&Lesbian":
        transformed_line = "Family-Sex&Relationships"
    elif transformed_line == "Science&Nature" or transformed_line == "Pets":
        transformed_line = "Science&Nature"
    elif transformed_line == "Children" or transformed_line == "Teens":
        transformed_line = "Children&Teens"
        
    return Row(id=line.id, familyid=line.familyid, itemid=line.itemid, categoryid=line.categoryid, slug=line.slug, name=transformed_line, languageid=line.languageid, votes = line.votes)

#NOTA: slug è rimasto invariato nel caso servisse il genere originale da cui è stata fatta l'aggregazione (nel dubbio)

RDDgenres_mapped = RDDgenres.map(mapGenres)
DFgenres = RDDgenres_mapped.toDF([])

#Fase 2.2: aggreghiamo i nuovi generi duplicati sommando i voti (partitionBy)

DFgenres = DFgenres.select("*", F.sum(DFgenres.votes).over(Window.partitionBy(DFgenres.itemid, DFgenres.name)).alias("new_votes")).drop(DFgenres.votes)
DFgenres = DFgenres.withColumnRenamed("new_votes", "votes").dropDuplicates(["itemid", "name"])

#DFgenres.filter(DFgenres.itemid == "10000108").show(3, False)

#Fase 3: nel caso di libri con più di 3 generi, tagliare facendo rimanere solo i primi 3 generi per voti
DFgenres = DFgenres.withColumn("rank", row_number().over(Window.partitionBy(DFgenres.itemid).orderBy(col("votes").desc()))).filter(col("rank") <= 3)
#DFgenres.show(20, False)

#Fase 4: aggregazione dei libri con più genere sotto un'unica tupla contenente la lista dei suoi generi 
#(segnarsi i voti di ogni genere se servisse). Torniamo agli RDD

DFgenres = DFgenres.select(concat_ws("/", DFgenres.name, DFgenres.votes).alias("genre_with_votes"), "id", "familyid", "itemid", "categoryid", "languageid", "slug")
#DFgenres.show(5000, False)

RDDgenres = DFgenres.rdd
RDDgenres_pair = RDDgenres.map(lambda x: (x.itemid, x))

def aggregateGenres(line1, line2):
    return Row(id=line1.id, familyid=line1.familyid, itemid=line1.itemid, categoryid=line1.categoryid, slug=line1.slug, languageid=line1.languageid, genre_with_votes = line1.genre_with_votes + " " + line2.genre_with_votes)

RDDgenres_reduced = RDDgenres_pair.reduceByKey(aggregateGenres)

def toSingleGenres(line1):
    return Row(id=line1[1].id, familyid=line1[1].familyid, itemid=line1[1].itemid, categoryid=line1[1].categoryid, slug=line1[1].slug, languageid=line1[1].languageid, genre_with_votes = line1[1].genre_with_votes)

RDDgenres_single = RDDgenres_reduced.map(toSingleGenres)

def toDictGenre(line1):
    dict_genres = {}
    genre_vote_list = line1.genre_with_votes.split(" ")
    for genre_vote in genre_vote_list:
        genre = genre_vote.split("/")[0]
        vote = genre_vote.split("/")[1]
        dict_genres[genre] = vote

    return Row(id=line1.id, familyid=line1.familyid, itemid=line1.itemid, categoryid=line1.categoryid, slug=line1.slug, languageid=line1.languageid, genre_with_votes = dict_genres)

RDDgenres_single = RDDgenres_single.map(toDictGenre)
DFgenres = RDDgenres_single.toDF([])



In [40]:
##Concateniamo i contenuti delle descrizioni con quelli dei commenti##
from pyspark.sql import functions as F


#Filtriamo le recensioni con almeno... 100 parole? Da valutare
DFreviews = DFreviews.filter(F.size(F.split('comment_content', ' ')) >= 120)

    

In [41]:
#DFfilteredmagazineitems.show(20, False)

#FILTRAGGIO 4: libri con più di X votes. Per vedere i voti di ogni libro dovrei usare il dataframe link_person_item.csv.
#Come possiamo notare, alcune recensione hanno valore 0: questo non vuol dire che il libro è stato valutato
#atrocemente ma che l'utente ha letto il libro senza inserire voti.

#Dobbiamo dunque scremare tali voti da questo dataframe.

# file = "/data/SMARTDATA/books/anobii/link_person_item.csv" #MOSTRA IL VOTO DI UN UTENTE A UN LIBRO
# DFstars = spark.read.csv(file,header=True)
#DFstars.show(50, False)

DFstarsfilteredno0 = DFinfo.filter(DFinfo.item_review > 0)
#DFstarsfilteredno0.show(20,False)

#Contiamo i libri con più voti e visualizziamoli in ordine discendente

DFgrouped = DFstarsfilteredno0.groupby("item_id").count().withColumnRenamed("count", "total_votes")\
                              .sort("total_votes", ascending=False)
#I voti dei libri più votati si aggirano intorno ai 10^4 come ordine di grandezza per cui per ora scelgo 300 come
#soglia di voti necessaria a rimanere nel dataframe (soggetto a cambiamenti)
DFgroupedfiltered = DFgrouped.filter(DFgrouped['total_votes'] > 100) #NUOVA SOGLIA: 150 PER 2500 LIBRI ANOBII
DFjoinbookstars = DFfilteredmagazineitems.join(DFgroupedfiltered, DFfilteredmagazineitems.item_id == DFgroupedfiltered.item_id).drop(DFgroupedfiltered.item_id)
# DFjoinbookstars.show(1, False)
# DFjoinbookstars:
# +-------+----------+-----------------+---------+----------+----------------+---------+---------+-------+----------------+------------+------------+------------------+--------+--------------+-----------+
# |item_id|isbn      |title            |sub_title|no_of_page|publication_date|publisher|binding  |edition|average_rating  |total_review|product_type|encrypt_item_id   |language|total_wishlist|total_votes|
# +-------+----------+-----------------+---------+----------+----------------+---------+---------+-------+----------------+------------+------------+------------------+--------+--------------+-----------+
# |1981748|8845260372|Giulietta squeenz|null     |209       |2008-05-01      |Bompiani |Paperback|null   |3.18773234200743|129         |1           |014f24ec9629744c88|11      |110           |543        |
# +-------+----------+-----------------+---------+----------+----------------+---------+---------+-------+----------------+------------+------------+------------------+--------+--------------+-----------+

#FILTRAGGIO 5: Pulizie varie ed eventuali
#DFfilteredlessthan10items.filter(DFfilteredlessthan10items.total_review > 100).show()

#NOTA: prendo i titoli con le minuscole per facilità di merging con le biblioteche
DFmanifestations_definitive_anobii = DFjoinbookstars.select("item_id", \
                                                            lower(DFjoinbookstars.title), \
                                                            lower(DFjoinbookstars.sub_title), \
                                                            "isbn",\
                                                            "average_rating", \
                                                            "total_review", \
                                                            "total_wishlist", \
                                                            "no_of_page",
                                                            "publication_date",
                                                            "publisher",
                                                            "binding",
                                                            "edition",
                                                            "product_type",
                                                            "total_votes",
                                                            "encrypt_item_id",)\
                                                    .withColumnRenamed("lower(title)", "title")\
                                                    .withColumnRenamed("lower(sub_title)", "sub_title")
# DFmanifestations_definitive_anobii.show(1, False)
# DFmanifestations_definitive_anobii: (uguale a DFjoinbookstars ma senza language)
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+
# |item_id|title            |sub_title|isbn      |average_rating  |total_review|total_wishlist|no_of_page|publication_date|publisher|binding  |edition|product_type|total_votes|encrypt_item_id   |
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+
# |1981748|giulietta squeenz|null     |8845260372|3.18773234200743|129         |110           |209       |2008-05-01      |Bompiani |Paperback|null   |1           |543        |014f24ec9629744c88|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+

print("Anobii data have been filtered (select books in italian, drop comics and books with a low number of ratings)")
print("Integrete with genre and author attributes Anobii data")
#add genres to the item table: this is contained in DFgenres:
# +---+--------+-------+----------+------------------+------------------+----------+-----+
# |id |familyid|itemid |categoryid|slug              |name              |languageid|votes|
# +---+--------+-------+----------+------------------+------------------+----------+-----+
# |1  |32423317|2823074|3         |business-economics|Business&Economics|3         |2.0  |
# +---+--------+-------+----------+------------------+------------------+----------+-----+
# Notice that if a book has more than 1 genre, DFgenres contained more than one row with that item_id, one for genre.
# Aggregation of rows with same itemid is needed 
#DFgenres_aggregated = DFgenres.groupBy('itemid')\
#                              .agg(F.concat_ws(" / ", F.collect_list('name'))\
#                                    .alias('genres'))
# DFgenres_aggregated:
# +-------+----------------------------------------------------------------------------------------------+
# |itemid |genres                                                                                        |
# +-------+----------------------------------------------------------------------------------------------+
# |1981748|Humor / Fiction&Literature / Romance / Family-Sex&Relationships / Teens / Philosophy / History|
# +-------+----------------------------------------------------------------------------------------------+

# join between items and DFgenres_aggregated to have, for each item, its genres
DFmanifestations_definitive_anobii_genres = DFmanifestations_definitive_anobii.join(DFgenres,\
                                                                                    DFgenres.itemid == DFmanifestations_definitive_anobii.item_id)\
                                                                              .select(DFmanifestations_definitive_anobii["*"],\
                                                                                      DFgenres.genre_with_votes) 

# DFmanifestations_definitive_anobii_genres:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+
# |item_id|title            |sub_title|isbn      |average_rating  |total_review|total_wishlist|no_of_page|publication_date|publisher|binding  |edition|product_type|total_votes|encrypt_item_id   |genres                                                                                        |
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+
# |1981748|giulietta squeenz|null     |8845260372|3.18773234200743|129         |110           |209       |2008-05-01      |Bompiani |Paperback|null   |1           |543        |014f24ec9629744c88|History / Fiction&Literature / Family-Sex&Relationships / Teens / Humor / Romance / Philosophy|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+
# add a column which explain the origin of the data (anobii or BCT)
DFmanifestations_definitive_anobii_genres = DFmanifestations_definitive_anobii_genres.withColumn('data_type', lit("anobii"))
# DFmanifestations_definitive_anobii_genres.show(1, False)
# DFmanifestations_definitive_anobii_genres:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+---------+
# |item_id|title            |sub_title|isbn      |average_rating  |total_review|total_wishlist|no_of_page|publication_date|publisher|binding  |edition|product_type|total_votes|encrypt_item_id   |genres                                                                                        |data_type|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+---------+
# |1981748|giulietta squeenz|null     |8845260372|3.18773234200743|129         |110           |209       |2008-05-01      |Bompiani |Paperback|null   |1           |543        |014f24ec9629744c88|Family-Sex&Relationships / Teens / Humor / Romance / Philosophy / History / Fiction&Literature|anobii   |
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+---------+
# add author in the DFmanifestations_definitive_anobii_genres table
# find author_id for each book
DFmanifestations_anobii_genres_author = DFmanifestations_definitive_anobii_genres.join(DFauthorbooks, \
                                                                                       DFauthorbooks.item_id == DFmanifestations_definitive_anobii_genres.item_id)\
                                                                                 .select(DFmanifestations_definitive_anobii_genres["*"], \
                                                                                         DFauthorbooks["author_id"])
# DFmanifestations_anobii_genres_author.show(1, False)
# DFmanifestations_anobii_genres_author:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+---------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|author_id|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+---------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Romance / Humor /...|   anobii|   354644|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+---------+
# find author (name) for each book (and remove books which has same title but different auhtor?)
DFmanifestations_anobii_genres_author = DFmanifestations_anobii_genres_author.join(DFdisplay,\
                                                                                   DFmanifestations_anobii_genres_author.author_id == DFdisplay.author_id)\
                                                                             .select(DFmanifestations_anobii_genres_author["*"], \
                                                                                     DFdisplay["author_name"])\
                                                                             .drop("author_id")

# remove books which has same title but different auhtor
DFmanifestations_anobii_genres_author = DFmanifestations_anobii_genres_author.select("*", F.min(DFmanifestations_anobii_genres_author.author_name)\
                                                                                           .over(Window.partitionBy(DFmanifestations_anobii_genres_author.item_id))\
                                                                                           .alias("author"))\
                                                                             .drop("author_name")\
                                                                             .dropDuplicates(['item_id', 'author'])
# DFmanifestations_anobii_genres_author.show(1, False)
# DFmanifestations_anobii_genres_author:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|    author|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Philosophy / Hist...|   anobii|Pulsatilla|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+
#join of item and rating tables to obtain person_id, item_id, rating and all attributes of the item in the same row
#df1.join(df2, df1.id == df2.id).select(df1["*"],df2["other"])
DFloans_definitive_anobii_genres = DFstarsfilteredno0.join(DFmanifestations_anobii_genres_author, \
                                                           DFmanifestations_anobii_genres_author.item_id == DFstarsfilteredno0.item_id)\
                                                     .select(DFmanifestations_anobii_genres_author["*"], \
                                                             DFstarsfilteredno0["person_id"], \
                                                             DFstarsfilteredno0["item_review"])
# DFloans_definitive_anobii_genres.show(1, False)
# DFloans_definitive_anobii_genres:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|    author|person_id|item_review|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Philosophy / Fict...|   anobii|Pulsatilla|  1244593|          3|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+

print("Anobii data have been integreted with genre and author attributes")
print("Read and filter BCT data")
%run cleanDataBCTwithIBSN.ipynb #Così a quanto pare
DFmanifestations_definitive_bct = DFmanifestations_definitive
DFloans_definitive_bct = DFloans_definitive
# DFmanifestations_definitive_bct.show(1, False)
# DFmanifestations_definitive_bct:
# +----------------+------------+---------+-------------+---------+--------------------+------------+
# |edition_language|edition_date|    title|       author|publisher|manifestation_id_new|ISBNISSN_new|
# +----------------+------------+---------+-------------+---------+--------------------+------------+
# |             ita|        1996|Pinocchio|Carlo Collodi|   Nuages|              107930|  8807820714|
# +----------------+------------+---------+-------------+---------+--------------------+------------+
# DFloans_definitive_bct:
# +--------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
# |manifestation_id_new|       patron_id_md5|    loan_date_begin|      loan_date_end|           due_date|from_library|to_library|end_library|
# +--------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
# |              220771|fa242f1458100dccc...|2012-09-05 11:27:25|2012-09-27 18:18:00|2012-10-05 11:27:21|          18|        18|         18|
# +--------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
print("BCT data have been read and filtered")
print("Prepare data for merge")

In [42]:
#from pyspark.sql.types import MapType

# BCT data preparation: prepare the table of loans which contains also metadata of the book
DFloans_with_titles = DFloans_definitive_bct.join(DFmanifestations_definitive_bct, \
                                                  DFloans_definitive_bct.manifestation_id_new == DFmanifestations_definitive_bct.manifestation_id_new)\
                                             .select(DFloans_definitive_bct.manifestation_id_new, \
                                                     "patron_id_md5", \
                                                     "author", \
                                                     "ISBNISSN_new", \
                                                     lower(DFmanifestations_definitive_bct.title))\
                                             .withColumnRenamed("lower(title)", "title")
# DFloans_with_titles:
# +--------------------+--------------------+-------------+------------+---------+
# |manifestation_id_new|       patron_id_md5|       author|ISBNISSN_new|    title|
# +--------------------+--------------------+-------------+------------+---------+
# |              107930|b5c0986c79b1afafd...|Carlo Collodi|  8807820714|pinocchio|
# +--------------------+--------------------+-------------+------------+---------+

# BCT data preparation: split title and subtitle
RDDloans_with_titles = DFloans_with_titles.rdd    
RDDloans_for_merge = RDDloans_with_titles.map(title_and_subtitle)  
# DFloans_with_titles.show(1, False)
DFloans_to_merge = RDDloans_for_merge.toDF(["item_id", \
                                            "person_id", \
                                            "title", \
                                            "author", \
                                            "sub_title", \
                                            "item_review", \
                                            "total_review", \
                                            "average_rating", \
                                            "total_votes", \
                                            "isbn", \
                                            "data_type"])
# BCT data preparation: add missing columns, which are present in DFloans_definitive_anobii_genres
columns_to_add = ['total_wishlist',
                  'no_of_page',
                  'publication_date',
                  'publisher',
                  'binding',
                  'edition',
                  'product_type',
                  'encrypt_item_id',
                  'genre_with_votes',
                 ]
values_to_add = {'total_wishlist': None,
                  'no_of_page': None,
                  'publication_date': None, 
                  'publisher': None, 
                  'binding': "Paperback",
                  'edition': None,
                  'product_type': 1,
                  'encrypt_item_id': None,
                  'genre_with_votes': None,
    
}
for c in columns_to_add:
        DFloans_to_merge = DFloans_to_merge.withColumn(c, lit(values_to_add[c]))

DFloans_to_merge = DFloans_to_merge.select("item_id", "title", "sub_title", "isbn", "average_rating", "total_review", "total_wishlist", "no_of_page", "publication_date", "publisher", "binding", "edition", "product_type", "total_votes", "encrypt_item_id", "genre_with_votes", "data_type", "author", "person_id", "item_review")

#DFloans_to_merge = DFloans_to_merge.withColumn("genre_with_votes",DFloans_to_merge.genre_with_votes.cast(MapType(StringType(), StringType())))
#DFloans_definitive_anobii_genres.printSchema()
#DFloans_to_merge.printSchema()

# DFloans_to_merge:
# +-------+--------------------+---------+-----------+-------------+-----------+------------+--------------+-----------+----------+-----------+--------------+----------+----------------+---------+---------+-------+------------+---------------+-----+
# |item_id|           person_id|    title|author_name|    sub_title|item_review|total_review|average_rating|total_votes|      isbn|  data_type|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|encrypt_item_id|genre|
# +-------+--------------------+---------+-----------+-------------+-----------+------------+--------------+-----------+----------+-----------+--------------+----------+----------------+---------+---------+-------+------------+---------------+-----+
# | 107930|fcc17388ef20567fe...|pinocchio|       None|Carlo Collodi|          4|        None|          None|       None|8807820714|biblioteche|          null|      null|            null|     null|Paperback|   null|           1|           null| null|
# +-------+--------------------+---------+-----------+-------------+-----------+------------+--------------+-----------+----------+-----------+--------------+----------+----------------+---------+---------+-------+------------+---------------+-----+
print("Data have been prepared for merge")
print("Merge data")
# union of the loans of bct data and anobii data
DFloans_merged = DFloans_definitive_anobii_genres.union(DFloans_to_merge)
# DFloans_merged.show(1)
# DFloans_merged
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|    author|person_id|item_review|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Family-Sex&Relati...|   anobii|Pulsatilla|   734393|          5|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# Remove books, whose title is one word only, i.e. keep books whose title contains at least a blank space
# DFloans_merged.filter(DFloans_merged.title == "pinocchio").show()
DFloans_merged_nosinglewordtitles = DFloans_merged.filter(DFloans_merged.title.contains(" ")) 
# DFloans_merged_nosinglewordtitles.filter(DFloans_merged_nosinglewordtitles.title == "pinocchio").show()

# Assign to books with the same title, the same book_id
DFloans_merged_aggregated = DFloans_merged_nosinglewordtitles.select("*", \
                                                                     F.first(DFloans_merged_nosinglewordtitles.item_id)\
                                                                      .over(Window.partitionBy(DFloans_merged_nosinglewordtitles.title).orderBy(DFloans_merged_nosinglewordtitles.data_type))\
                                                                      .alias("book_id"))\
                                                              .drop("item_id")

Data have been prepared for merge
Merge data


In [170]:
#DFloans_merged_aggregated.show(2000, False)

In [43]:
# DFloans_merged_aggregated.show(1)
# DFloans_merged_aggregated:
# +--------------------+-------------------+----------+----------------+------------+--------------+----------+----------------+-------------------+---------+-------+------------+-----------+------------------+--------------------+---------+----------------+---------+-----------+-------+
# |               title|          sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|          publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|          author|person_id|item_review|book_id|
# +--------------------+-------------------+----------+----------------+------------+--------------+----------+----------------+-------------------+---------+-------+------------+-----------+------------------+--------------------+---------+----------------+---------+-----------+-------+
# |come mi batte for...|storia di mio padre|8806198882|4.14122681883024|         206|           270|       302|      2009-11-03|Einaudi (Frontiere)|Hardcover|      1|           1|        705|01500978c278d06718|Crime / Teens / N...|   anobii|Benedetta Tobagi|   767565|          3|2806310|
# +--------------------+-------------------+----------+----------------+------------+--------------+----------+----------------+-------------------+---------+-------+------------+-----------+------------------+--------------------+---------+----------------+---------+-----------+-------+

# Assign to books with the same title, the same new_author
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.author)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_author"))

# Assign to books with the same title, the same new_genre
#DFloans_merged_aggregated.show(1, False)
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.first(DFloans_merged_aggregated.genre_with_votes)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id).orderBy(DFloans_merged_aggregated.data_type))\
                                                              .alias("new_genre"))

# Assign to books with the same title, the same new_encrypt
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.encrypt_item_id)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_encrypt_item_id"))

# Assign to books with the same title, the same new_isbn
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.isbn)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_isbn"))

# Update metrics of loans: new_total_count (to count the number of votes or loans) 
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.count("*")\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_total_count"))

# Update metrics of loans: new_average_rating 
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.avg(DFloans_merged_aggregated.item_review)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_average_rating"))

# Update metrics of loans: new_total_review, to count the actual number of review for a book (only anobii provides real review!!) 
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.max(DFloans_merged_aggregated.total_review)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_total_review"))

# Remove "old" attributes
columnsToDrop = ['author',
                 'genre_with_votes',
                 'encrypt_item_id',
                 'isbn',
                 'total_count',
                 'average_rating',
                 'total_review',]
DFloans_merged_aggregated = DFloans_merged_aggregated.drop(*columnsToDrop)
# DFloans_merged_aggregated.show(1)
# DFloans_merged_aggregated:
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+-------------------+----------+---------------+------------------+----------------+
# |            title|sub_title|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|data_type|person_id|item_review|book_id|new_author|           new_genre|new_encrypt_item_id|  new_isbn|new_total_count|new_average_rating|new_total_review|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+-------------------+----------+---------------+------------------+----------------+
# |giulietta squeenz|     null|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|   anobii|   959028|          3|1981748|Pulsatilla|Philosophy / Hist...| 014f24ec9629744c88|8845260372|            543|3.1860036832412524|             129|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+-------------------+----------+---------------+------------------+----------------+

# Rename "new" attributes
columnsToRenames = ['new_author',
                    'new_genre',
                    'new_encrypt_item_id',
                    'new_isbn',
                    'new_total_count',
                    'new_average_rating',
                    'new_total_review',]
for c in columnsToRenames:
    DFloans_merged_aggregated = DFloans_merged_aggregated.withColumnRenamed(c, c.replace("new_", ""))
# DFloans_merged_aggregated.show(1)
# DFloans_merged_aggregated:
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+------------------+----------+-----------+------------------+------------+
# |            title|sub_title|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|data_type|person_id|item_review|book_id|    author|               genre|   encrypt_item_id|      isbn|total_count|    average_rating|total_review|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+------------------+----------+-----------+------------------+------------+
# |giulietta squeenz|     null|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|   anobii|   273238|          4|1981748|Pulsatilla|Humor / Philosoph...|014f24ec9629744c88|8845260372|        543|3.1860036832412524|         129|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+------------------+----------+-----------+------------------+------------+

# Remove ['book_id', 'person_id'] duplicates
DFloans_merged_aggregated = DFloans_merged_aggregated.dropDuplicates(['book_id', 'person_id'])
print("Data has been merged")
# before write data on file, create index for both users and books of the sparce matrix
RDDmerged_almost_definitive = DFloans_merged_aggregated.rdd

# functions which creates the dictionary where the key is the book_id/person_id and the value is the corresponding index
book_dictionary = create_book_dictionary(RDDmerged_almost_definitive)
user_dictionary = create_user_dictionary(RDDmerged_almost_definitive)

# add the user_index and the book_index in the dataframe
RDDdefinitive = RDDmerged_almost_definitive.map(addRowColumnId)
DFloans_merged_aggregated = RDDdefinitive.toDF(['book_id',
                                                'title',
                                                'sub_title',
                                                'total_wishlist',
                                                'no_of_page',
                                                'publication_date',
                                                'publisher',
                                                'binding',
                                                'edition',
                                                'product_type',
                                                'total_votes',
                                                'data_type',
                                                'person_id',
                                                'item_review',
                                                'author',
                                                'genre',
                                                'encrypt_item_id',
                                                'isbn',
                                                'total_count',
                                                'average_rating',
                                                'total_review', 
                                                'user_index', 
                                                'book_index'], sampleRatio=0.9)
print("Write data on csv files")

Data has been merged
Write data on csv files


In [200]:
#DFloans_merged_aggregated.show(2000, False)

In [44]:
ratings = DFloans_merged_aggregated.filter(~(DFloans_merged_aggregated.genre.isNull()))#DFdefinitive
print(ratings.count())
# in the rating table, each row is given by "person_id", "book_id", "rating" attributes
ratings_tocsv = ratings.select(["person_id", "book_id", "item_review"])\
                       .withColumnRenamed("item_review", "rating")
# ratings_tocsv.show(1)
# ratings_tocsv:
# +---------+-------+------+
# |person_id|book_id|rating|
# +---------+-------+------+
# |   100489|1981748|     3|
# +---------+-------+------+
print("There are "+str(ratings_tocsv.count())+" loans")
ratings_tocsv.toPandas().to_csv("ratings2021_extended.csv")

# in the book table, the metadata about the book are reported

5987892
There are 5987892 loans


In [45]:
# +-------+-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+----------+--------------------+------------------+----------+-----------+------------------+------------+----------+
# |book_id|            title|sub_title|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|data_type|    author|               genre|   encrypt_item_id|      isbn|total_count|    average_rating|total_review|book_index|
# +-------+-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+----------+--------------------+------------------+----------+-----------+------------------+------------+----------+
# |1981748|giulietta squeenz|     null|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|   anobii|Pulsatilla|Family-Sex&Relati...|014f24ec9629744c88|8845260372|        543|3.1860036832412524|         129|       319|
# +-------+-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+----------+--------------------+------------------+----------+-----------+------------------+------------+----------+


books_tocsv = ratings.dropDuplicates(['book_id'])\
                     .drop("person_id")\
                     .drop("user_index")\
                     .drop("item_review")
# books_tocsv.show(1)
# books_tocsv:
print("There are "+str(books_tocsv.count())+" books")
books_tocsv = books_tocsv.join(DFdescriptions, DFdescriptions.item_id == books_tocsv.book_id).select("book_id", "title", "sub_title", "total_wishlist", "no_of_page", "publication_date", "publisher", "binding", "edition", "product_type", "total_votes", "data_type", "author", "genre", "encrypt_item_id", "isbn", "total_count", "average_rating", "total_review", "book_index", "content")
#books_tocsv.toPandas().to_csv("books2021_extended.csv")
#books_tocsv.show(1, False)
books_tocsv = books_tocsv.withColumn("content2", books_tocsv["content"])
print(books_tocsv.select("book_id").distinct().count())

RDDbooks_tocsv = books_tocsv.rdd
RDDbooks_tocsv = RDDbooks_tocsv.map(lambda x: (x.book_id, x))
RDDreducedbooks = RDDbooks_tocsv.reduceByKey(concatenateContentsDescriptions)
RDDreducedbooks = RDDreducedbooks.map(toSingleDescriptions)
books_tocsv = RDDreducedbooks.toDF([])
print(books_tocsv.count())

#Joiniamo DFreviews a books_tocsv (la cardinalità aumenterà ma con una funzione di riduzione con gli RDD dovremmo riportarla alla normalità)
books_tocsv = books_tocsv.join(DFreviews, books_tocsv.book_id == DFreviews.item_id, "left").select(books_tocsv.book_id, "title", "sub_title", "total_wishlist", "no_of_page", "publication_date", "publisher", "binding", "edition", "product_type", "total_votes", "data_type", "author", "genre", "encrypt_item_id", "isbn", "total_count", "average_rating", "total_review", "book_index", "content", "comment_content", "content2")

#Fatto ciò,  trasformiamo books_tocsv in un rdd per poter applicare la funzione di concatenazione
print(books_tocsv.select("book_id").distinct().count())

RDDbooks = books_tocsv.rdd
RDDbooks = RDDbooks.map(lambda x: (x.book_id, x))
RDDreducedbooks = RDDbooks.reduceByKey(concatenateContentsComments)
RDDreducedbooks = RDDreducedbooks.map(toSingleComments)
RDDreducedbooks = RDDreducedbooks.map(mergeDescriptionComments)
books_tocsv = RDDreducedbooks.toDF([])

print(books_tocsv.count())

# in the user table, the person_id is reported for each user (in each row)
users_tocsv = ratings.select("person_id", "user_index").dropDuplicates()
# users_tocsv.show(1)
# users_tocsv:
# +---------+----------+
# |person_id|user_index|
# +---------+----------+
# |  1224803|     10623|
# +---------+----------+
print("There are "+str(users_tocsv.count())+" users")
users_tocsv.toPandas().to_csv("users2021_extended.csv")

print("Csv files have been written")
#books_tocsv.show(1, False)

There are 9198 books
3582
3582
3582
3582
There are 231646 users
Csv files have been written


In [46]:
###ESTRAZIONE KEYWORD###

#Usiamo una versione filtrata di books_tocsv dove non compaiono tuple senza descrizione per attuare la nostra
#estrazione.

books_with_content = books_tocsv.filter(books_tocsv.content!="")


In [47]:
books_with_content = books_with_content.filter(~(books_with_content.content.contains("Scheda INCOMPLETA"))) #Filtra i libri con le schede incomplete
books_with_content = books_with_content.filter(~(books_with_content.content.contains("SCHEDA DOPPIA")))
books_with_content = books_with_content.filter(~(books_with_content.content.contains("Questa scheda è DOPPIA.")))
books_with_content = books_with_content.filter(~(((books_with_content.content.contains("scheda")) | (books_with_content.content.contains("Scheda"))) & ((books_with_content.content.contains("incompleta")) | (books_with_content.content.contains("doppia")))))
#books_with_content = books_with_content.dropDuplicates(['book_id'])

#books_with_content.filter(books_with_content.content.contains("Esistt")).show(1, False)

#Per estrarre le keyword in maniera "TF-IDF" dobbiamo innanzitutto implementare una funzione che pulisca il testo 
#dalle stopwords (congiunzioni, rumori, etc.)

def clean_text_light(doc_collection):
    new_corpus_doc = []
    for description in doc_collection:
        description = description.replace("agrave", " ")
        description = description.replace("egrave", " ")
        description = description.replace("igrave", " ")
        description = description.replace("ograve", " ")
        description = description.replace("ograve", " ")
        description = description.replace("<b>", " ")
        description = description.replace(",", " ")
        description = description.replace("'", " ")
        description = description.replace('"', " ")
        description = description.replace("<b>", " ")
        description = description.replace("<br />", " ")
        description = description.replace("</b>", " ")
        description = description.replace("<br>", " ")
        description = description.replace("</br>", " ")
        description = description.replace("<p>", " ")
        description = description.replace("</p>", " ")
        description = description.replace("<P>", " ")
        description = description.replace("</P>", " ")
        description = description.replace("<i>", " ")
        description = description.replace("</i>", " ")
        description = description.replace("&quot", " ")
        description = description.replace("<strong>", " ")
        description = description.replace("</strong>", " ")
        description = description.replace("&", " ")  
        #KEYWORD PRESENTI MA NON MOLTO UTILI
        pattern = re.compile(re.escape("leggere"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("lettrice"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("lettore"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("lettori"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("lettura"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("libro"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("libri"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" mese "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" mesi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("tempo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("tempi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("parte "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("parti "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" cosa "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" cose "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" personaggio "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" personaggi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" volume "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" volumi"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" protagonista"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" protagonisti"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" testo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" testi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" pagina "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" pagine "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" secolo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" secoli "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" persona "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" persone "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("scrittore "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" scrittrice "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" scrittrici "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" scrittori "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        #pattern = re.compile(re.escape(" vic"), re.IGNORECASE)
        #description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" fine "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("nome "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("nomi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("inizio "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" parola "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" parole "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        #pattern = re.compile(re.escape(" ista"), re.IGNORECASE)
        #description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("modo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" numero "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" punto "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" centro "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        #pattern = re.compile(re.escape("ora "), re.IGNORECASE)
        #description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" frattempo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" mezzo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" mezzi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" corso "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" situazione "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" piano "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" via "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" vie "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" forma "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" forme "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" posto "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" posti "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" fronte "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" luogo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("autore "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" autori "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" autrici "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("autrice "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" grazie "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" fatto "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" fatti "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("edizione "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" edizioni "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" opere "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" serie "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" tema "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" temi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" grado "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" gradi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" genere "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" generi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" titolo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" titoli "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("oggetto "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" oggetti "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" tempo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" tempi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" sfondo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" sfondi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" conto "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" conti "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" volta "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" volte "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        #pattern = re.compile(re.escape(" race"), re.IGNORECASE)
        #description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" scena "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" scene "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("età "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" figura "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" figure "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("epoca "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" epoche"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("none"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("libreria"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("librerie"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("biblioteca"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("biblioteche"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("generazione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("generazioni"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" mano "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" mani "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("aspetto "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("titolo"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("produzione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" senso "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" trama "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" trame "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("racconto"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("racconti"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("capitolo"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("capitoli"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" fondo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("domanda"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("risposta"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("domande"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("risposte"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("versione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("argomento"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("argomenti"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" tesi "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("italiano"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("effetto"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" carta "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("bisogno"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("bisogni"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("analisi"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("momento"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("milione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("disegno"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" dono "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("successo"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("traduzione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" penna "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("regola"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("regole"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("progetto"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("esempio"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("narrazione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" data "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("pratica"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("frase"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("ultimo"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("base"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("effetto"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("attenzione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("motivo"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("essere"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("linguaggio"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" lingua "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("lavoro"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("critica"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("aspetto"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("introduzione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("episodio"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("livello"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("pubblicazione"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape(" pezzo "), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("interesse"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("consiglio"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        pattern = re.compile(re.escape("effetto"), re.IGNORECASE)
        description = pattern.sub(' ', description)
        #pattern = re.compile(re.escape("ore"), re.IGNORECASE)
        #description = pattern.sub(' ', description)
       
        new_corpus_doc.append(description)
    return new_corpus_doc

In [48]:
DFcontent_pandas = books_with_content.toPandas()

corpus_doc = DFcontent_pandas['content'].to_list()
#print(corpus_doc[0])
cleaned_corpus_doc = clean_text_light(corpus_doc) #Pulizia rumori
#print(cleaned_corpus_doc[0])

In [49]:
###FILTRIAMO NOMI E AGGETTIVI DALLE DESCRIZIONI CON SPACY###
##CELLA DI ESEMPIO PER L'UTILIZZO DI SPACY, UN MODULO PER CLASSIFICARE IL RUOLO DELLE PAROLE NELE FRASI
import spacy

nlp = spacy.load("it_core_news_lg") #Caricamento file per il riconoscimento

new_corpus_doc = []
for document in cleaned_corpus_doc:
    noun_tokens = []
    doc = nlp(document)
    for token in doc:
        if token.pos_ == "NOUN" and token.is_oov==False: #Solo nomi presenti nel vocabolario italiano
            noun_tokens.append(token.lemma_)
    new_document = (" ").join(noun_tokens)
    new_corpus_doc.append(new_document)

#print(new_corpus_doc[:10])

In [10]:
#CELLA DI PROVA PER SPACY

#text = "programmando è proprio bello"
#doc = nlp(text)
#for token in doc:
#    print(token.text, token.pos_, token.dep_, token.is_oov, token.lemma_)

programmando VERB advcl False programmare
è AUX cop False essere
proprio ADV advmod False proprio
bello ADJ ROOT False bello


In [14]:
###FASE DI STEMMING### #SOSTITUITA A FASE DI LEMMATIZING, NON USARE

#Fase usata per unire le parole chiave molto simili fra loro

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

# import stemmer
from nltk.stem import SnowballStemmer
#Crea uno stemmer italiano
stemmer_snowball = SnowballStemmer('italian')
stop_words = set(stopwords.words('italian'))

stemmed_corpus_doc = []
for descrizione in new_corpus_doc:
    new_tokens_without_sw = []
    text_token = word_tokenize(descrizione)
    tokens_without_sw = [word for word in text_token if not word in stop_words]
    
    for word in tokens_without_sw:
        stem_word = stemmer_snowball.stem(word)
        new_tokens_without_sw.append(stem_word)
    filtered_sentence = (" ").join(new_tokens_without_sw)
    
    stemmed_corpus_doc.append(filtered_sentence)

#print(stemmed_corpus_doc[:10])

In [83]:
#Importiamo il TFIDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True, max_df=0.35, min_df=80) #Di solito 0.60, 40

vectorizer.fit_transform(new_corpus_doc) #CREAZIONE DIZIONARIO CON I VOCABOLI CONTENUTI NELLE DESCRIZIONI

feature_names = vectorizer.get_feature_names() #Estrae le feature (i vocaboli) importanti

def sort_coo(coo_matrix): #Funzione usata per riordinare il dizionario per score
    tuples = zip(coo_matrix.col, coo_matrix.data) #COLONNE: FEATURES RIGHE: DESCRIZIONE
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=5): #Funzione usata per ottenere le n parole chiave
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
    
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

def get_keywords(vectorizer, feature_names, doc): #Funzione usata per ritornare le top k parole chiave di una descrizione
    tf_idf_vector = vectorizer.transform([doc])
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords=extract_topn_from_vector(feature_names,sorted_items, 20) #10 keyword per ora
    
    return list(keywords.keys())

result = []
for descrizione1, descrizione2 in zip(corpus_doc, new_corpus_doc):
    df = {}
    df['content'] = descrizione1
    df['top_keywords'] = get_keywords(vectorizer, feature_names, descrizione2)
    result.append(df)
    
DFkeywords = pd.DataFrame(result)
DFkeywords = DFkeywords[DFkeywords['top_keywords'].str.len() > 0]
DFkeywords

/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1448: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


,content,top_keywords
0,"Kimihiro Watanuki, liceale, possiede la capaci...","[negozio, desidero, strega, spirito, proprieta..."
1,"Dopo anni di duro apprendistato, nel 1976 Carv...","[normalità, marito, sensazione, fotografia, mo..."
2,"Per il commissario Vittorio Spotorno, il dupli...","[commissario, gioco, vista, meta, quartiere, m..."
3,"Parigi, giugno 1889. La città palpita, travolt...","[proprietario, onda, nipote, tecnica, città, m..."
4,Leona Dorn &egrave; soddisfatta della sua vita...,"[suicidio, casa, fiducia, incubo, giornalista,..."
...,...,...
3527,"È il destino, ancora una volta, a dare le cart...","[notte, luce, moglie, fede, medico, matrimonio..."
3528,"Lauren torna a Wisteria, Maryland, sette anni ...","[madre, incidente, zia, colpevole, morte, pres..."
3529,Ciclo Dirk Pitt - Vol. 19Dirk Pitt resta quasi...,"[avventura, ricerca, squadra, nave, tocco, ond..."
3530,"Don Fabrizio, principe di Salina, all'arrivo d...","[felicità, pretesto, sopravvivenza, decina, af..."


In [84]:
#Ora che abbiamo il dataframe di pandas con le keywords associate alle descrizioni, non ci resta che joinare
#il contenuto di esso con il dataframe dei libri, integrando l'attributo keywords.
#Riportiamo il dataframe nella forma PySpark

DFkeywords = spark.createDataFrame(DFkeywords)

DFkeywords = DFkeywords.dropDuplicates(['content'])
DF_almost_final = books_with_content.join(DFkeywords, DFkeywords.content == books_with_content.content).drop(DFkeywords.content).select("book_id", "top_keywords")

#Adesso joiniamo per book_id al dataframe con più libri (compresi quelli senza descrizione)
DF_final = books_with_content.join(DF_almost_final, DF_almost_final.book_id == books_with_content.book_id).drop(DF_almost_final.book_id)

In [85]:
DF_final = DF_final.filter(F.size(col("top_keywords")) > 3) #Scartiamo i libri con poche info come parole chiave
#print(DF_final.count())

In [86]:
#Proviamo a inserire nel dataframe il link per l'immagine della copertina seguendo lo schema del link di anobii:
#https://media.anobii.com/covers/id/width/height/scale/treatment/format/version

#Ovviamente, il link deve essere costruito solo per le tuple che hanno un "encrypt item id" altrimenti sarà null
#Per ora metto il link con dei parametri "safe": immagine 250x250, formato png, no timestamp, scale 1, treatment = paperback

#NOTA: In caso poi si possono cambiare i parametri

def createImageUrl(line):
    if line.encrypt_item_id == None: #Non c'è un item_encrypt_id
        return line
    else: #C'è l'id
        image_url = f"https://media.anobii.com/covers/{line.encrypt_item_id}/250/250/1/paperback/png/"     
        return Row(book_id=line.book_id, title=line.title, sub_title=line.sub_title, 
               total_wishlist=line.total_wishlist, no_of_page=line.no_of_page, publication_date=line.publication_date, 
               publisher=line.publisher, binding=line.binding, edition=line.edition, product_type=line.product_type, 
               total_votes=line.total_votes, data_type=line.data_type, author=line.author, genre=line.genre, 
               encrypt_item_id=line.encrypt_item_id, isbn=line.isbn, total_count=line.total_count, average_rating=line.average_rating, 
               total_review=line.total_review, book_index=line.book_index, content=line.content, description = line.content2, top_keywords=line.top_keywords, image_url=image_url)
    
#Aggiungiamo la nuova colonna al dataframe books_tocsv.
RDDfinal = DF_final.rdd
RDDfinal_with_urls = RDDfinal.map(createImageUrl)

DF_final = RDDfinal_with_urls.toDF()

In [87]:
#print(DF_final.count()) #3495 LIBRI
#print(DF_final.select('book_id').distinct().count())
from pyspark.sql.functions import array_contains

DF_final = DF_final.withColumn('content', regexp_replace('content', '\,', ''))
DF_final = DF_final.withColumn('content2', regexp_replace('content', '\,', ''))
DF_final = DF_final.withColumn('content', regexp_replace('content', "\'", ''))
DF_final = DF_final.withColumn('content2', regexp_replace('content', "\'", ''))
DF_final = DF_final.withColumn('content', regexp_replace('content', '\"', ''))
DF_final = DF_final.withColumn('content2', regexp_replace('content', '\"', ''))
DF_final.toPandas().to_csv('for_now_final_DF5.csv')

#DF_final.filter(array_contains(col("top_keywords"), "scheda")).show(20, False)

In [88]:
###KEYWORD COVERAGE###

##In questa sezione proverò a creare un set di N keywords (quante? Da decidere, per ora facciamo 100, anche se non
##è detto che il problema abbia soluzione con 100) che coprano tutto il dataset dei 3500 libri.

#Non avendo una chiara idea su come procedere, tecnicamente parlando, proverò a risolvere il problema in maniera
#"creativa". Creiamo, innanzitutto, un dizionario contenente le parole chiave estratte dal dataframe DF_final
#e assegniamo un "peso" ad ogni parola chiave. Il peso, nel nostro caso, non sarà altro che il numero di occorrenze
#della parola chiave nella lista: infatti, se una parola compare, ad esempio, 5 volte, vuol dire che la parola chiave
#"copre" 5 libri e così via. Questo vuol dire che, sapendo che il nostro dataframe ha 3554 libri, accumulare
#3554 unità di peso dei vocaboli dovrebbe garantire una coverage completa del dataset.

#Iniziamo con il contare i termini con il countvectorizer


keywords_list = DF_final.toPandas()['top_keywords'].to_list()

flat_list = [keyword for keywords_sublist in keywords_list for keyword in keywords_sublist]
#print(len([item for item in flat_list if item=="storia"]))
#print(flat_list)

#Adesso che abbiamo la lista dei vari vocaboli, creiamo un dizionario che contenga elementi nella forma
#vocabolo: occorrenza.

occurrence_dict = {}

def countOccurrencies(dictionary, reference_list):
    for keyword in reference_list:
        if keyword in dictionary:
            dictionary[keyword] += 1
        elif keyword not in dictionary:
            dictionary[keyword] = 1
    return dictionary

occurrence_dict = countOccurrencies(occurrence_dict, flat_list)
sorted_dict = {k: v for k, v in sorted(occurrence_dict.items(), key=lambda item: item[1], reverse=True)}
print(sorted_dict)

{'amore': 498, 'donna': 489, 'casa': 362, 'amico': 342, 'figlio': 342, 'famiglia': 333, 'morte': 313, 'giorno': 311, 'po': 300, 'ragazzo': 296, 'bambino': 287, 'vicenda': 284, 'guerra': 280, 'madre': 280, 'padre': 269, 'caso': 259, 'viaggio': 240, 'realtà': 228, 'paese': 224, 'città': 222, 'avventura': 220, 'moglie': 206, 'ragazza': 203, 'passato': 203, 'notte': 188, 'ricerca': 186, 'potere': 181, 'sogno': 178, 'idea': 178, 'occhio': 175, 'passione': 172, 'cuore': 171, 'marito': 169, 'saga': 162, 'stile': 160, 'strada': 160, 'fratello': 159, 'indagine': 154, 'forza': 153, 'rapporto': 153, 'terra': 152, 'sentimento': 151, 'opera': 149, 'mistero': 149, 'segreto': 148, 'corpo': 145, 'destino': 145, 'mente': 144, 'omicidio': 143, 'sangue': 143, 'pensiero': 140, 'esistenza': 139, 'dolore': 138, 'voce': 138, 'finale': 137, 'ricordo': 136, 'scrittura': 134, 'capo': 134, 'socio': 133, 'signore': 132, 'giovane': 132, 'letteratura': 132, 'paura': 132, 'film': 131, 'problema': 129, 'genitore': 12

In [83]:
from pyspark.sql.functions import array_contains, col
#DF_final.show(2000)

In [89]:
#Il dizionario ha adesso una forma più decente.

#Implementiamo una funzione che estragga un set di keywords che coprano l'intero dataset di libri.

#La funzione deve darci un set di X keywords (per ora facciamo 100) che:
#-Copra tutto il dataset
#-Abbia keywords il meno ridondanti possibile

#Abbiamo a disposizione il dataframe (DF_final) e il dizionario (sorted_dict).


#import random
#DF_final.filter(array_contains(col('top_keywords'), 'entusiasmo')).show(10, False) #Possiamo usare array_contains per filtrare
#per keyword
def keywordCoverage(dictionary, dataframe):
    
    output_list = []
    
    temp_dictionary = dictionary.copy()
    temp_subset = dataframe.toPandas()['top_keywords'].to_list()
    
    while True:
        
        max_coverage_count = 0
        chosen_keyword = ""
        
        for keyword in temp_dictionary:
        
            coverage_count = len([item for item in temp_subset if keyword in item])
            if coverage_count > max_coverage_count:
                max_coverage_count = coverage_count
                chosen_keyword = keyword
            
        try:
            choice = input(f"La parola scelta è {chosen_keyword}. Ha una coverage di {max_coverage_count}. La lunghezza rimanente del dataset è {len(temp_subset)}. (y/n)")

            if choice == "y":
                temp_dictionary.pop(chosen_keyword)
                output_list.append(chosen_keyword)
                temp_subset = [item for item in temp_subset if chosen_keyword not in item]
                print("KEYWORD SCELTA")
                if len(temp_subset) == 0:
                    break
                    
            elif choice == "n":
                temp_dictionary.pop(chosen_keyword)
                print("KEYWORD SCARTATA")
                if len(temp_subset) == 0:
                    break
            else:
                raise ValueError

        except ValueError:
            print("Scegliere una scelta possibile (y/n).")

    return output_list

chosen_N = keywordCoverage(sorted_dict, DF_final)
print(chosen_N)
print(len(chosen_N))
            
                    

La parola scelta è amore. Ha una coverage di 498. La lunghezza rimanente del dataset è 3427. (y/n) y


KEYWORD SCELTA


La parola scelta è donna. Ha una coverage di 356. La lunghezza rimanente del dataset è 2929. (y/n) y


KEYWORD SCELTA


La parola scelta è amico. Ha una coverage di 251. La lunghezza rimanente del dataset è 2573. (y/n) y


...

KEYWORD SCELTA


La parola scelta è fotografia. Ha una coverage di 1. La lunghezza rimanente del dataset è 1. (y/n) n


KEYWORD SCARTATA


La parola scelta è entusiasmo. Ha una coverage di 1. La lunghezza rimanente del dataset è 1. (y/n) y


KEYWORD SCELTA
['amore', 'donna', 'amico', 'guerra', 'caso', 'bambino', 'morte', 'ragazzo', 'viaggio', 'famiglia', 'realtà', 'avventura', 'paese', 'idea', 'città', 'padre', 'indagine', 'problema', 'ragazza', 'capo', 'sogno', 're', 'arte', 'ricerca', 'signore', 'giornalista', 'sangue', 'paura', 'cultura', 'occhio', 'gioco', 'raccolta', 'passato', 'tragedia', 'nemico', 'esistenza', 'delitto', 'sistema', 'battaglia', 'lotta', 'genio', 'memoria', 'film', 'macchina', 'scienza', 'notte', 'ordine', 'assassino', 'riflessione', 'studio', 'saga', 'scuola', 'animale', 'diritto', 'acqua', 'quartiere', 'scontro', 'morto', 'poesia', 'classico', 'male', 'ombra', 'orrore', 'artista', 'cronaca', 'inglese', 'compito', 'sentimento', 'mistero', 'destino', 'avvocato', 'bar', 'semplicità', 'figlia', 'stato', 'vento', 'commedia', 'moglie', 'omicidio', 'mare', 'studente', 'notizia', 'chiesa', 'ciclo', 'narratore', 'passione', 'rapporto', 'polizia', 'amicizia', 'lettera', 'popolo', 'compagno', 'relazione', 'cr

In [9]:
#Dizionario keyword scelte : percentuale di generi
from pyspark.sql.functions import col, array_contains, size
from pyspark.sql.functions import array
from pyspark.sql.functions import split
from pyspark.sql.functions import regexp_replace
import numpy as np
import json
import pandas as pd

#Oltre alla nostra lista delle keyword, dovremo usare il solito DF_final.
#print(chosen_N)
DF_final = spark.read.csv('anobii_genres/for_now_final_DF4.csv', header=True)
DF_final = DF_final.withColumn('top_keywords', regexp_replace('top_keywords', '\[', ''))
DF_final = DF_final.withColumn('top_keywords', regexp_replace('top_keywords', '\]', ''))
DF_final = DF_final.withColumn('top_keywords', regexp_replace('top_keywords', "\'", ''))
DF_final = DF_final.withColumn('top_keywords', regexp_replace('top_keywords', "\'", ''))
DF_final = DF_final.withColumn('top_keywords', regexp_replace('top_keywords', "\, ", ','))
#DF_final_new = DF_final_new.withColumn('genre_vector', array(DF_final_new.genre_vector))

DF_final = DF_final.withColumn('top_keywords', split(col('top_keywords'), ","))

#DF_final.show(1, False)
#LISTA GIà CREATA
chosen_N = ['amore', 'donna', 'amico', 'guerra', 'caso', 'bambino', 'morte', 'ragazzo', 'viaggio', 'famiglia', 'realtà', 'avventura', 'paese', 'idea', 'città', 'padre', 'indagine', 'problema', 'ragazza', 'capo', 'sogno', 're', 'arte', 'ricerca', 'signore', 'giornalista', 'sangue', 'paura', 'cultura', 'occhio', 'gioco', 'raccolta', 'passato', 'tragedia', 'nemico', 'esistenza', 'delitto', 'sistema', 'battaglia', 'lotta', 'genio', 'memoria', 'film', 'macchina', 'scienza', 'notte', 'ordine', 'assassino', 'riflessione', 'studio', 'saga', 'scuola', 'animale', 'diritto', 'acqua', 'quartiere', 'scontro', 'morto', 'poesia', 'classico', 'male', 'ombra', 'orrore', 'artista', 'cronaca', 'inglese', 'compito', 'sentimento', 'mistero', 'destino', 'avvocato', 'bar', 'semplicità', 'figlia', 'stato', 'vento', 'commedia', 'moglie', 'omicidio', 'mare', 'studente', 'notizia', 'chiesa', 'ciclo', 'narratore', 'passione', 'rapporto', 'polizia', 'amicizia', 'lettera', 'popolo', 'compagno', 'relazione', 'crisi', 'fantascienza', 'vicino', 'cielo', 'angelo', 'gioia', 'entusiasmo']
#chosen_N = ['amore', 'donna', 'amico', 'morte', 'bambino', 'guerra', 'caso', 'realtà', 'ragazzo', 'figlio', 'paese', 'città', 'idea', 'avventura', 'famiglia', 'viaggio', 'mistero', 'problema', 'ricerca', 'potere', 'raccolta', 'paura', 'polizia', 'padre', 'delitto', 'arte', 'saga', 'giornalista', 'passato', 'film', 'battaglia', 'gioco', 'universo', 'notte', 'capo', 'ragazza', 'pensiero', 'movimento', 'cuore', 'male', 'tragedia', 'speranza', 'morto', 'droga', 'processo', 'animale', 'poesia', 'genio', 'signore', 'mare', 'killer', 'coppia', 'riflessione', 'scoperta', 'aiuto', 'faccia', 'psicologia', 'esistenza', 'cultura', 'dio', 'studio', 'cronaca', 'voce', 'artista', 'trattato', 'assassino', 'natura', 'cane', 'caccia', 'civiltà', 'fantascienza', 'quadro', 'moglie', 'passione', 'strada', 'figlia', 'provincia', 'sangue', 'orrore', 'compagno', 'stato', 'sfida', 'ordine', 'angelo', 'voglia', 'oro', 'rischio', 'rivelazione', 'rapporto', 'sentimento', 'lettera', 'vittima', 'amante', 'relazione', 'testa', 'popolo', 'comunità', 'ciclo', 'ispirazione', 'entusiasmo']
def genresPercentage(dataframe, _list):
    
    DF_final_pandas = dataframe.toPandas() #Local dataframe
    percentage_dict = {}
    
    for keyword in _list:
        masking = DF_final_pandas.top_keywords.apply(lambda x: keyword in x)
        DF_final_keyword = DF_final_pandas[masking] #Dataframe contenente solo le tuple che contengono la keyword

        genre_list = DF_final_keyword['genre'].to_list() #Estraiamo il dizionario con i generi per ogni tupla con la keyword

        percentage_dict[keyword] = {'total': len(genre_list)}

        for dictionary in genre_list:
            dictionary = dictionary.replace("'", '"')
            dictionary = json.loads(dictionary)
            
            for key in dictionary:

                if key in percentage_dict[keyword]:
                    percentage_dict[keyword][key] += float(dictionary[key])/sum(list(map(float, dictionary.values()))) #WEIGHTED

                else:
                    
                    percentage_dict[keyword][key] = float(dictionary[key])/sum(list(map(float, dictionary.values())))

        for key in percentage_dict[keyword]:

            if key != 'total':
                percentage_dict[keyword][key] = (percentage_dict[keyword][key] * 100) / percentage_dict[keyword]['total']
    return percentage_dict
        
percentage_dict = genresPercentage(DF_final, chosen_N)
#print(percentage_dict)

In [10]:
dataframe = pd.DataFrame(percentage_dict)
dataframe

,amore,donna,amico,guerra,caso,bambino,morte,ragazzo,viaggio,famiglia,...,popolo,compagno,relazione,crisi,fantascienza,vicino,cielo,angelo,gioia,entusiasmo
total,498.000000,489.000000,342.000000,280.000000,259.000000,287.000000,313.000000,296.000000,240.000000,333.000000,...,89.000000,82.000000,79.000000,76.000000,74.000000,66.000000,66.000000,59.000000,57.000000,41.000000
Family-Sex&Relationships,7.758273,7.133577,5.301741,2.326511,2.359581,3.998856,3.538663,4.220919,2.891508,6.951478,...,0.517999,4.824721,7.936964,5.496743,0.979787,6.267687,3.729795,2.423222,5.420215,4.339015
Fiction&Literature,46.598101,47.166554,37.712174,42.023418,32.562166,41.243915,40.017384,38.401293,41.063557,49.739449,...,38.565369,39.538734,43.094314,40.440970,30.012201,51.801614,42.426208,36.569484,41.567252,38.862795
Humor,1.683975,2.261595,3.121255,1.015098,2.531946,3.252735,1.004468,1.507328,1.953963,1.633051,...,2.382013,2.026766,3.448042,5.670057,1.173463,2.712446,2.140362,2.473487,2.112668,2.428131
History,4.160573,6.304667,2.380553,15.590335,2.919778,4.292048,5.502305,2.635927,4.481042,7.038948,...,16.719551,6.863171,3.768243,4.696467,0.568990,4.888728,4.470452,1.986855,4.735335,6.656105
ScienceFiction&Fantasy,12.291724,5.681256,12.896563,13.440439,5.663632,9.829995,13.780516,18.786913,15.399738,7.331449,...,14.831851,15.611158,8.393392,3.268926,44.355780,7.831203,14.729790,21.061625,3.074400,1.668250
Travel,1.042892,0.477669,1.331200,1.606718,0.508792,1.013002,0.607691,0.572132,9.378677,0.423487,...,5.377636,2.963852,1.170361,1.276208,0.080766,0.151515,1.699078,0.039417,0.985276,1.251837
Romance,9.485980,6.754494,4.912996,1.618118,1.670488,2.064946,2.555626,3.790989,1.437224,4.333231,...,0.692088,2.679235,6.677591,4.857566,0.344828,2.983254,3.968895,5.380706,9.871348,6.033089
Non-fiction,0.862230,0.973327,1.112126,0.669858,1.676658,0.572411,1.327439,0.515594,1.492492,0.481186,...,0.741651,0.503049,0.309264,1.751363,1.452137,0.145903,0.793814,0.500770,1.012127,2.151197
Biography,1.800709,3.056844,2.215511,3.715312,1.168251,2.875914,2.784120,1.031910,2.795534,3.442409,...,4.828882,1.948300,1.785902,1.130707,0.616406,0.479798,1.415149,0.459228,2.577250,5.070610


In [11]:
#Nota: eliminare tutti i libri con genere Comics&GraphicNovels ? Da valutare

#Adesso che abbiamo il dataframe del peso delle varie keyword, possiamo creare il vettore "generi" dei libri sui
#quali calcolare la similarità.

#VETTORI GENERI = [0,0,....,0]
#DFgenres.show(200, False)
#Il vettore genere è lungo 21 elementi e contiene, nella i-esima posizione, un valore che indica quanto il libro
#appartiene a quel genere.

mapping = {'Fiction&Literature': 0,
              'Family-Sex&Relationships': 1,
              'Humor': 2,
              'History': 3,
              'ScienceFiction&Fantasy': 4,
              'Romance': 5,
              'Travel': 6,
              'Mystery&Thrillers': 7,
              'FreeTime': 8,
              'Non-fiction': 9,
              'Biography': 10,
              'SocialScience': 11,
              'Political': 12,
              'Crime': 13,
              'Children&Teens': 14,
              'Philosophy': 15,
              'Horror': 16,
              'Health-Mind&Body': 17,
              'Professional&Technical': 18,
              'Science&Nature': 19,
              'Comics&GraphicNovels': 20}

DF_final_pandas = DF_final.toPandas()

DF_mapped = pd.Series(DF_final_pandas['genre'].values)

def mapSeries(line):
    
    line = line.replace("'", '"')
    line = json.loads(line)
    for key in line:
        line[key] = float(line[key])
        
    total = sum(line.values())
    genre_vector = [0 for i in range(21)]
    
    for key in line:
        value_genre = line[key] / total
        genre_vector[mapping[key]] = value_genre
    
    return genre_vector

DF_transformed = DF_mapped.map(mapSeries)

DF_final_pandas_new = DF_final_pandas.assign(genre_vector = DF_transformed)
DF_final_pandas_new['genre_vector'] = DF_final_pandas_new['genre_vector'].astype(str)

DF_final_new = spark.createDataFrame(DF_final_pandas_new)

DF_final_new = DF_final_new.withColumn('genre_vector', regexp_replace('genre_vector', '\[', ''))
DF_final_new = DF_final_new.withColumn('genre_vector', regexp_replace('genre_vector', '\]', ''))
#DF_final_new = DF_final_new.withColumn('genre_vector', array(DF_final_new.genre_vector))

DF_final_new = DF_final_new.withColumn('genre_vector', split(col('genre_vector'), ","))

#NOTA: eliminare comics? Come migliorare le keywords?


In [14]:
#DF_final_new = DF_final_new.drop('_c0').drop('content2')

DF_final_new.toPandas().to_csv('for_now_final_DF_with_genre_vector.csv')


In [1]:
#DF_final = spark.read.csv('for_now_final_DF_with_genre_vector.csv', header=True)
#DF_final.show(1, False)

In [14]:
DF_final = spark.read.csv("for_now_final_DF_with_genre_vector.csv", header=True)
#DF_final.show(1, False)

In [15]:
#CREAZIONE FILE JSON SECONDO I CANONI DEL TEAM VR

import json
import pandas as pd

DF_final_pandas = DF_final.toPandas()
#print(type(DF_final_pandas['genre_vector']))
#DF_final_pandas.groupby(['book_id']).apply(lambda x: x[['genre_vector']].to_dict()).reset_index().to_json(orient='records')

#DF_final_pandas.to_json('books.json')
#JSON_ = DF_final_pandas.to_dict('records')
#print(JSON_[0:5])
column_list = []
column_keywords_list =  []
column_dict_out = []

for genre_string in DF_final_pandas['genre_vector']:
    
    genre_string = genre_string.replace("[","").replace("]","").replace("'","")
    genre_list = [float(element.replace("  ", "")) for element in genre_string.split(",")]
    column_list.append(genre_list)  
    
for keywords_string in DF_final_pandas['top_keywords']:
    keywords_string = keywords_string.replace("[","").replace("]","").replace("'","")
    keywords_list = [str(element.replace(" ", "")) for element in keywords_string.split(",")]
    column_keywords_list.append(keywords_list)
    
for genre_string_dict in DF_final_pandas['genre']: 
    column_dict = []
    genre_string_dict = genre_string_dict.replace("'", '"')
    genre_dict = json.loads(genre_string_dict)
    for key in genre_dict:
        genre_string = key + ":" + genre_dict[key]
        column_dict.append(genre_string)
    column_dict_out.append(column_dict)

#print(column_dict_out)
DF_final_pandas['genre_vector'] = pd.Series(column_list).values
DF_final_pandas['top_keywords'] = pd.Series(column_keywords_list).values
DF_final_pandas['genre'] = pd.Series(column_dict_out).values



In [29]:

#DF_final_pandas.groupby(['book_id']).apply(lambda x: x[['genre_vector']].to_dict('records')).reset_index().to_json(orient='records')
#DF_final_pandas.to_json('books.json')

dict_ = DF_final_pandas.to_dict()
#3426 books

#Rearrange the json
output_dict = [] #Containing elements with the structure "book_id" : "metadata"
#print(output_dict)
with open('books3.json', 'w') as f:
    for _id in range(3426):
        metadata_dict = {}
        for key in dict_:
            metadata_dict[key] = dict_[key][_id]
            #print(metadata_dict)
        #print(metadata_dict['genre'])
        output_dict.append(metadata_dict)
    json.dump([output_dict], f)
        
    
        
#print(output_dict[1])
#with open('books.json', 'w') as f:
#    json.dump([], f)